# Image Digit Classification

In [1]:
from keras import Model
from keras. layers import Input, Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from keras.callbacks import EarlyStopping
import pandas as pd
import numpy as np

from PIL import Image, ImageDraw

## Image Processing
- Reshape 1D arrays into 2D arrays.
- Separate image data and labels into distinct variables (processed_images, labels).

In [2]:
df = pd.read_csv("train_mnist.csv")
processed_images = []
labels = []
columns = df.columns
for index,row in df.iterrows():
    size = 28
    image = []
    label = row["label"]
    for i in range(size):
        image_row = []
        for j in range(size):
            image_row.append(0)
        image.append(image_row)

    for column in columns:
        splited_column = column.split("x")
        if not len(splited_column) == 2:
            continue
        x = int(splited_column[0])-1
        y = int(splited_column[1])-1
        image[x][y] = row[column]

    labels.append(label)
    processed_images.append(image)


## Image display function

In [3]:
import tkinter as tk

def showImage(label, image):
    root = tk.Tk()
    root.title(f"Number: {label}")

    img = tk.PhotoImage(width=28, height=28)

    pixels = ""
    for row in image:
        row_str = "{" + " ".join('#{:02x}{:02x}{:02x}'.format(int(v), int(v), int(v)) for v in row) + "} "
        pixels += row_str

    img.put(pixels)
    big_img = img.zoom(10)
    canvas = tk.Canvas(root, width=300, height=300)
    canvas.pack()

    canvas.create_image(150, 150, image=big_img)
    canvas.image = big_img

    root.mainloop()


### Example

In [4]:
index_to_show = 0
label = labels[index_to_show]
image_data = processed_images[index_to_show]

showImage(label, image_data)

## Model

In [5]:
X = np.array(processed_images)
X = np.expand_dims(X, axis=-1)
X = X.astype('float32') / 255.0

y = np.array(labels)

print(f"Images in dataset: {X.shape[0]}")
print(f"shape of each image: {X.shape[1:]}")

Images in dataset: 30000
shape of each image: (28, 28, 1)


In [6]:
input_layer = Input(shape=(X.shape[1:]))

conv2d1 = Conv2D(filters=4, kernel_size=(2, 2), strides=(1, 1), activation='relu')(input_layer)
pool2d1 = MaxPooling2D(pool_size=(2, 2))(conv2d1)

conv2d2 = Conv2D(filters=8, kernel_size=(2, 2), strides=(1, 1), activation='relu')(pool2d1)
pool2d2 = MaxPooling2D(pool_size=(2, 2))(conv2d2)

flatten = Flatten()(pool2d2)

fl_1 = Dense(300, activation="relu")(flatten)
drop1 = Dropout(0.2)(fl_1)
fl_2 = Dense(150, activation="relu")(drop1)
drop2 = Dropout(0.2)(fl_2)
fl_3 = Dense(50, activation="relu")(drop1)
drop3 = Dropout(0.2)(fl_3)

output_layer = Dense(10, activation="softmax")(drop3)

In [7]:
cnn2D_model = Model(inputs=input_layer, outputs=output_layer)
es = EarlyStopping(monitor="val_loss",  patience=5, restore_best_weights=True)
cnn2D_model.compile(optimizer="adam",
                    loss="sparse_categorical_crossentropy",
                    metrics=["accuracy"])
cnn2D_model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 28, 28, 1)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 27, 27, 4)      │            20 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 13, 13, 4)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 12, 12, 8)      │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 6, 6, 8)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 288)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 300)            │        86,700 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 300)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 50)             │        15,050 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 50)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 10)             │           510 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 102,416 (400.06 KB)

 Trainable params: 102,416 (400.06 KB)

 Non-trainable params: 0 (0.00 B)

In [8]:
history = cnn2D_model.fit(X, y, epochs=100, batch_size=32, validation_split=0.1, verbose=2, callbacks=[es])
print("finished training")

Epoch 1/100
844/844 - 6s - 7ms/step - accuracy: 0.8720 - loss: 0.4107 - val_accuracy: 0.9557 - val_loss: 0.1337
Epoch 2/100
844/844 - 4s - 5ms/step - accuracy: 0.9584 - loss: 0.1385 - val_accuracy: 0.9717 - val_loss: 0.0842
Epoch 3/100
844/844 - 4s - 4ms/step - accuracy: 0.9706 - loss: 0.0982 - val_accuracy: 0.9737 - val_loss: 0.0746
Epoch 4/100
844/844 - 4s - 5ms/step - accuracy: 0.9765 - loss: 0.0779 - val_accuracy: 0.9807 - val_loss: 0.0619
Epoch 5/100
844/844 - 4s - 5ms/step - accuracy: 0.9801 - loss: 0.0625 - val_accuracy: 0.9797 - val_loss: 0.0699
Epoch 6/100
844/844 - 5s - 6ms/step - accuracy: 0.9835 - loss: 0.0541 - val_accuracy: 0.9833 - val_loss: 0.0529
Epoch 7/100
844/844 - 5s - 6ms/step - accuracy: 0.9861 - loss: 0.0453 - val_accuracy: 0.9820 - val_loss: 0.0583
Epoch 8/100
844/844 - 5s - 6ms/step - accuracy: 0.9867 - loss: 0.0421 - val_accuracy: 0.9830 - val_loss: 0.0665
Epoch 9/100
844/844 - 5s - 6ms/step - accuracy: 0.9891 - loss: 0.0344 - val_accuracy: 0.9790 - val_loss:

## Canvas

In [9]:
import tkinter as tk
from PIL import Image, ImageDraw
import numpy as np

class DigitDrawer:
    def __init__(self):
        self.canvas_size = 280
        self.output_size = (28, 28)
        self.final_array = None

        self._image = Image.new("L", (self.canvas_size, self.canvas_size), 0)
        self._draw_context = ImageDraw.Draw(self._image)

    def draw(self):
        self.root = tk.Tk()
        self.root.title("Draw a Digit")

        self.canvas = tk.Canvas(self.root, width=self.canvas_size,
                                height=self.canvas_size, bg="black")
        self.canvas.pack()
        self.canvas.bind("<B1-Motion>", self._paint)

        btn_finish = tk.Button(self.root, text="Finish", command=self._finish)
        btn_finish.pack(fill=tk.X)

        self.root.mainloop()

    def _paint(self, event):
        r = 10  # Brush radius
        x1, y1 = (event.x - r), (event.y - r)
        x2, y2 = (event.x + r), (event.y + r)

        self.canvas.create_oval(x1, y1, x2, y2, fill="white", outline="white")
        self._draw_context.ellipse([x1, y1, x2, y2], fill=255)

    def _finish(self):
        # Resize to 28x28
        small_image = self._image.resize(self.output_size, resample=Image.Resampling.LANCZOS)
        self.final_array = np.array(small_image)

        self.root.destroy()

    def get(self):
        if self.final_array is None:
            print("Warning: No digit drawn yet. Call .draw() first.")
        return self.final_array


## Drawing and Prediction

In [10]:
def predict_number(my_digit):
    x = my_digit.reshape(1, 28, 28, 1)
    result = cnn2D_model.predict(x)
    predicted_digit = np.argmax(result)
    return predicted_digit

In [11]:
drawer = DigitDrawer()

In [12]:
drawer.draw()
my_digit = drawer.get()

if my_digit is not None:
    digit_pred = predict_number(my_digit)
    print("Predicted digit:", digit_pred)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 168ms/step
Predicted digit: 5
